In [2]:
import os
import typing
import numpy as np
import torch
import torch.optim
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, average_precision_score
from torch import nn
from torch.nn import functional as F
from tqdm import trange
import tqdm
from torch.distributions import Poisson
from collections import deque
import copy

from util import ece, ParameterDistribution, draw_reliability_diagram, draw_confidence_histogram, SGLD
from enum import Enum

from torch.utils.data import DataLoader
import json

In [3]:
class DECE(nn.Module):
    """
    Computes DECE loss (differentiable expected calibration error).
    """

    def __init__(self, device, num_bins, t_a, t_b):
        super(DECE, self).__init__()
        self.device = device
        self.num_bins = num_bins
        self.t_a = t_a
        self.t_b = t_b

    def one_hot(self, indices, depth):
        """
        Returns a one-hot tensor.
        This is a PyTorch equivalent of Tensorflow's tf.one_hot.
        Parameters:
        indices:  a (n_batch, m) Tensor or (m) Tensor.
        depth: a scalar. Represents the depth of the one hot dimension.
        Returns: a (n_batch, m, depth) Tensor or (m, depth) Tensor.
        """
        encoded_indicies = torch.zeros(
            indices.size() + torch.Size([depth])).to(device=self.device)
        index = indices.view(indices.size() + torch.Size([1]))
        encoded_indicies = encoded_indicies.scatter_(1, index, 1)

        return encoded_indicies

    def forward(self, input, target):
        if input.dim() > 2:
            # N,C,H,W => N,C,H*W
            input = input.view(input.size(0), input.size(1), -1)
            input = input.transpose(1, 2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))   # N,H*W,C => N*H*W,C

        # For CIFAR-10 and CIFAR-100, target.shape is [N] to begin with
        target = target.view(-1)

        predicted_probs = F.softmax(input, dim=1)

        cut_points = torch.linspace(0, 1, self.num_bins + 1)[:-1].to(device=self.device)
        W = torch.reshape(torch.linspace(1.0, self.num_bins, self.num_bins).to(device=self.device), [1, -1])
        b = torch.cumsum(-cut_points, 0)

        confidences = torch.max(predicted_probs, dim=1, keepdim=True)[0]
        h = torch.matmul(confidences, W) + b
        h = h / self.t_b

        bin_probs = F.softmax(h, dim=1)

        # smoothen the probabilities to avoid zeros
        eps = 1e-6
        bin_probs = bin_probs + eps
        # normalize the probabilities to sum to one across bins
        bin_probs = bin_probs / (1.0 + (self.num_bins + 1) * eps)

        # calculate bin confidences
        bin_confs = torch.div(bin_probs.transpose(0, 1).matmul(confidences).view(-1),
                              torch.sum(bin_probs, dim=0))
        # all-pairs approach
        batch_pred_diffs = torch.unsqueeze(predicted_probs, dim=2) - torch.unsqueeze(predicted_probs, dim=1)
        # computing pairwise differences, i.e., Sij or Sxy
        if str(self.device) == 'cpu':
            gpu = False
        else:
            gpu = True
        # using {-1.0*} may lead to a poor performance when compared with the above way;
        batch_indicators = robust_sigmoid(torch.transpose(batch_pred_diffs, dim0=1, dim1=2), self.t_a, gpu)

        # get approximated rank positions, i.e., hat_pi(x)
        ranks_all = torch.sum(batch_indicators, dim=2) + 0.5
        # the ranks go from 1 to C, with 1 being the best rank
        true_ranks = ranks_all[torch.arange(ranks_all.size(0)), target]
        accs = F.relu(2.0 - true_ranks)
        bin_accs = torch.div(bin_probs.transpose(0, 1).matmul(accs).view(-1),
                                torch.sum(bin_probs, dim=0))

        # calculate overall ECE for the whole batch
        ece = torch.sum(torch.sum(bin_probs, dim=0) * torch.abs(bin_accs - bin_confs) / bin_probs.shape[0], dim=0)
        return ece

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# soft accuracy:
# parts of the code are taken from https://github.com/wildltr/ptranking, specifically 
# https://github.com/wildltr/ptranking/blob/master/ptranking/base/neural_utils.py

# soft binning:
# we have used parts of the code from https://github.com/wOOL/DNDT/blob/master/pytorch/demo.ipynb


class Robust_Sigmoid(torch.autograd.Function):
    ''' Aiming for a stable sigmoid operator with specified sigma '''

    @staticmethod
    def forward(ctx, input, sigma=1.0, gpu=False):
        '''
        :param ctx:
        :param input: the input tensor
        :param sigma: the scaling constant
        :return:
        '''
        x = input if 1.0 == sigma else sigma * input

        torch_half = torch.cuda.FloatTensor(
            [0.5]) if gpu else torch.FloatTensor([0.5])
        sigmoid_x_pos = torch.where(
            input > 0, 1./(1. + torch.exp(-x)), torch_half)

        exp_x = torch.exp(x)
        sigmoid_x = torch.where(input < 0, exp_x/(1.+exp_x), sigmoid_x_pos)

        grad = sigmoid_x * \
            (1. - sigmoid_x) if 1.0 == sigma else sigma * \
            sigmoid_x * (1. - sigmoid_x)
        ctx.save_for_backward(grad)

        return sigmoid_x

    @staticmethod
    def backward(ctx, grad_output):
        '''
        :param ctx:
        :param grad_output: backpropagated gradients from upper module(s)
        :return:
        '''
        grad = ctx.saved_tensors[0]

        bg = grad_output * grad  # chain rule

        return bg, None, None


#- function: robust_sigmoid-#
robust_sigmoid = Robust_Sigmoid.apply

In [5]:
class Framework(object):
    def __init__(self, dataset_train:torch.utils.data.Dataset, *args, **kwargs):
        """
        Basic Framework for your bayesian neural network.
        Other solutions like MC Dropout, Ensemble learning will based upon this.
        """
        self.train_set = dataset_train
        self.print_interval = 100 # number of batches until updated metrics are displayed during training

    def train(self):
        raise NotImplementedError()

    def predict(self, data_loader: torch.utils.data.DataLoader) -> np.ndarray:
        """
        Predict the class probabilities using your trained model.
        This method should return an (num_samples, 10) NumPy float array
        such that the second dimension sums up to 1 for each row.

        :param data_loader: Data loader yielding the samples to predict on
        :return: (num_samples, 10) NumPy float array where the second dimension sums up to 1 for each row
        """
        probability_batches = []
        
        for batch_x, _ in tqdm.tqdm(data_loader):
            current_probabilities = self.predict_probabilities(batch_x).detach().numpy()
            probability_batches.append(current_probabilities)

        output = np.concatenate(probability_batches, axis=0)
        return output

    def predict_probabilities(self, x: torch.Tensor) -> torch.Tensor:
        raise NotImplementedError()

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dece = DECE(device, 30, 100, 0.01)

In [7]:
def evaluate(model:Framework, eval_loader: torch.utils.data.DataLoader, data_dir: str, output_dir: str):
    """
    Evaluate your model.
    :param model: Trained model to evaluate
    :param eval_loader: Data loader containing the training set for evaluation
    :param data_dir: Data directory from which additional datasets are loaded
    :param output_dir: Directory into which plots are saved
    """
    print("evaulating")
    # Predict class probabilities on test data
    predicted_probabilities = model.predict(eval_loader)

    # Calculate evaluation metrics
    predicted_classes = np.argmax(predicted_probabilities, axis=1)
    actual_classes = eval_loader.dataset.tensors[1].detach().cpu().numpy()
    accuracy = np.mean((predicted_classes == actual_classes)) 
    ece_score = ece(predicted_probabilities, actual_classes)
    print(f'Accuracy: {accuracy.item():.3f}, ECE score: {ece_score:.3f}')
    
    # TODO: Reliability_diagram_3. draw reliability diagram on Test Data
    # You can uncomment the below code to make it run. You can learn from
    # the graph about how to improve your model. Remember first to complete 
    # the function of calc_calibration_curve.
  
    # print('Plotting reliability diagram on Test Dataset')
    # out = calc_calibration_curve(predicted_probabilities, actual_classes, num_bins = 30)
    # fig = draw_reliability_diagram(out)
    # fig.savefig(os.path.join(output_dir, 'reliability-diagram.pdf'))
    EXTENDED_EVALUATION = False
    if EXTENDED_EVALUATION:
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
        ### draw reliability diagram
        print('Plotting reliability diagram')
        reliability_diagram_datax = np.load(os.path.join(data_dir, 'reliability_diagram_data_x.npy'))
        reliability_diagram_datay = np.load(os.path.join(data_dir, 'reliability_diagram_data_y.npy'))
        for i in range(3):
            demo_prob_i = reliability_diagram_datax[i]
            demo_label_i = reliability_diagram_datay[i]
            out = calc_calibration_curve(demo_prob_i, demo_label_i, num_bins = 9)
            fig = draw_reliability_diagram(out)
            fig.savefig(os.path.join(output_dir, str(i)+'_reliability-diagram.pdf'))
        
        eval_samples = eval_loader.dataset.tensors[0].detach().cpu().numpy()

        # Determine confidence per sample and sort accordingly
        confidences = np.max(predicted_probabilities, axis=1)
        sorted_confidence_indices = np.argsort(confidences)

        # Plot samples your model is most confident about
        print('Plotting most confident MNIST predictions')
        most_confident_indices = sorted_confidence_indices[-10:]
        fig, ax = plt.subplots(4, 5, figsize=(13, 11))
        for row in range(0, 4, 2):
            for col in range(5):
                sample_idx = most_confident_indices[5 * row // 2 + col]
                ax[row, col].imshow(np.reshape(eval_samples[sample_idx], (28, 28)), cmap='gray')
                ax[row, col].set_axis_off()
                ax[row + 1, col].set_title(f'predicted {predicted_classes[sample_idx]}, actual {actual_classes[sample_idx]}')
                bar_colors = ['C0'] * 10
                bar_colors[actual_classes[sample_idx]] = 'C1'
                ax[row + 1, col].bar(
                    np.arange(10), predicted_probabilities[sample_idx], tick_label=np.arange(10), color=bar_colors
                )
        fig.suptitle('Most confident predictions', size=20)
        fig.savefig(os.path.join(output_dir, 'mnist_most_confident.pdf'))

        # Plot samples your model is least confident about
        print('Plotting least confident MNIST predictions')
        least_confident_indices = sorted_confidence_indices[:10]
        fig, ax = plt.subplots(4, 5, figsize=(13, 11))
        for row in range(0, 4, 2):
            for col in range(5):
                sample_idx = least_confident_indices[5 * row // 2 + col]
                ax[row, col].imshow(np.reshape(eval_samples[sample_idx], (28, 28)), cmap='gray')
                ax[row, col].set_axis_off()
                ax[row + 1, col].set_title(f'predicted {predicted_classes[sample_idx]}, actual {actual_classes[sample_idx]}')
                bar_colors = ['C0'] * 10
                bar_colors[actual_classes[sample_idx]] = 'C1'
                ax[row + 1, col].bar(
                    np.arange(10), predicted_probabilities[sample_idx], tick_label=np.arange(10), color=bar_colors
                )
        fig.suptitle('Least confident predictions', size=20)
        fig.savefig(os.path.join(output_dir, 'mnist_least_confident.pdf'))

        print('Plotting ambiguous and rotated MNIST confidences')
        ambiguous_samples = torch.from_numpy(np.load(os.path.join(data_dir, 'test_x.npz'))['test_x']).reshape([-1, 784])[:10]
        ambiguous_dataset = torch.utils.data.TensorDataset(ambiguous_samples, torch.zeros(10))
        ambiguous_loader = torch.utils.data.DataLoader(
            ambiguous_dataset, batch_size=10, shuffle=False, drop_last=False
        )
        ambiguous_predicted_probabilities = model.predict(ambiguous_loader)
        ambiguous_predicted_classes = np.argmax(ambiguous_predicted_probabilities, axis=1)
        fig, ax = plt.subplots(4, 5, figsize=(13, 11))
        for row in range(0, 4, 2):
            for col in range(5):
                sample_idx = 5 * row // 2 + col
                ax[row, col].imshow(np.reshape(ambiguous_samples[sample_idx], (28, 28)), cmap='gray')
                ax[row, col].set_axis_off()
                ax[row + 1, col].set_title(f'predicted {ambiguous_predicted_classes[sample_idx]}')
                ax[row + 1, col].bar(
                    np.arange(10), ambiguous_predicted_probabilities[sample_idx], tick_label=np.arange(10)
                )
        fig.suptitle('Predictions on ambiguous and rotated MNIST', size=20)
        fig.savefig(os.path.join(output_dir, 'ambiguous_rotated_mnist.pdf'))


        # Do the same evaluation as on MNIST also on FashionMNIST
        print('Predicting on FashionMNIST data')
        fmnist_samples = torch.from_numpy(np.load(os.path.join(data_dir, 'fmnist.npz'))['x_test'])#.reshape([-1, 784])
        fmnist_dataset = torch.utils.data.TensorDataset(fmnist_samples, torch.zeros(fmnist_samples.shape[0]))
        fmnist_loader = torch.utils.data.DataLoader(
            fmnist_dataset, batch_size=64, shuffle=False, drop_last=False
        )
        fmnist_predicted_probabilities = model.predict(fmnist_loader)
        fmnist_predicted_classes = np.argmax(fmnist_predicted_probabilities, axis=1)
        fmnist_confidences = np.max(fmnist_predicted_probabilities, axis=1)
        fmnist_sorted_confidence_indices = np.argsort(fmnist_confidences)

        # Plot FashionMNIST samples your model is most confident about
        print('Plotting most confident FashionMNIST predictions')
        most_confident_indices = fmnist_sorted_confidence_indices[-10:]
        fig, ax = plt.subplots(4, 5, figsize=(13, 11))
        for row in range(0, 4, 2):
            for col in range(5):
                sample_idx = most_confident_indices[5 * row // 2 + col]
                ax[row, col].imshow(np.reshape(fmnist_samples[sample_idx], (28, 28)), cmap='gray')
                ax[row, col].set_axis_off()
                ax[row + 1, col].set_title(f'predicted {fmnist_predicted_classes[sample_idx]}')
                ax[row + 1, col].bar(
                    np.arange(10), fmnist_predicted_probabilities[sample_idx], tick_label=np.arange(10)
                )
        fig.suptitle('Most confident predictions', size=20)
        fig.savefig(os.path.join(output_dir, 'fashionmnist_most_confident.pdf'))

        # Plot FashionMNIST samples your model is least confident about
        print('Plotting least confident FashionMNIST predictions')
        least_confident_indices = fmnist_sorted_confidence_indices[:10]
        fig, ax = plt.subplots(4, 5, figsize=(13, 11))
        for row in range(0, 4, 2):
            for col in range(5):
                sample_idx = least_confident_indices[5 * row // 2 + col]
                ax[row, col].imshow(np.reshape(fmnist_samples[sample_idx], (28, 28)), cmap='gray')
                ax[row, col].set_axis_off()
                ax[row + 1, col].set_title(f'predicted {fmnist_predicted_classes[sample_idx]}')
                ax[row + 1, col].bar(
                    np.arange(10), fmnist_predicted_probabilities[sample_idx], tick_label=np.arange(10)
                )
        fig.suptitle('Least confident predictions', size=20)
        fig.savefig(os.path.join(output_dir, 'fashionmnist_least_confident.pdf'))

        print('Determining suitability of your model for OOD detection')
        all_confidences = np.concatenate([confidences, fmnist_confidences])
        dataset_labels = np.concatenate([np.ones_like(confidences), np.zeros_like(fmnist_confidences)])
        print(
            'AUROC for MNIST vs. FashionMNIST OOD detection based on confidence: '
            f'{roc_auc_score(dataset_labels, all_confidences):.3f}'
        )
        print(
            'AUPRC for MNIST vs. FashionMNIST OOD detection based on confidence: '
            f'{average_precision_score(dataset_labels, all_confidences):.3f}'
        )

In [8]:
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.LazyConv2d(8, 3, 1)
        self.conv2 = nn.LazyConv2d(16, 3, 1)
        self.fc1 = nn.LazyLinear(128)
        self.fc2 = nn.LazyLinear(10)
        self.d2d = nn.Dropout2d(p=0.1)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        # x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.dropout(x,training=True,p=0.3)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x,training=True,p=0.3)
        x = self.fc2(x)
        # output = self.softmax(x)
        return x

In [9]:
class DropoutTrainer(Framework):
    def __init__(self, dataset_train,
                 *args, **kwargs):
        super().__init__(dataset_train, *args, **kwargs)

        # Hyperparameters and general parameters
        # TODO: MC_Dropout_4. Do experiments and tune hyperparameters
        self.batch_size = 64
        self.learning_rate = 1e-3
        self.num_epochs = 100
        torch.manual_seed(0) # set seed for reproducibility
        
        # TODO: MC_Dropout_1. Initialize the MC_Dropout network and optimizer here
        # You can check the Dummy Trainer above for intuition about what to do
        self.network = MNISTNet()
        self.train_loader = DataLoader(
            dataset_train, batch_size=self.batch_size, shuffle=True, drop_last=True
            )
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=self.learning_rate) 
        

    def train(self):
        self.network.train()
        self.network  = self.network.to("cuda")
        # self.train_loader  = self.train_loader.to("cuda")
        progress_bar = trange(self.num_epochs)
        # scheduler = ReduceLROnPlateau(optimizer, 'min')
        for epoch in progress_bar:
            
            if epoch % 50 == 0:
                torch.save(self.network.state_dict(),f"trainer_{epoch}.pth")

            for batch_idx, (batch_x, batch_y) in enumerate(self.train_loader):
                # batch_x are of shape (batch_size, 784), batch_y are of shape (batch_size,)

                batch_x = batch_x.to("cuda").view(-1,1,28,28)
                batch_y = batch_y.to("cuda")

                self.network.zero_grad()
                # TODO: MC_Dropout_2. Implement MCDropout training here
                # You need to calculate the loss based on the literature
                preds = F.softmax(self.network(batch_x),dim=-1)
                # loss = F.nll_loss(preds,batch_y)
                ece = dece(preds, batch_y)
                loss = F.cross_entropy(preds, batch_y)
                loss = loss + ece*(epoch/np.sqrt(self.num_epochs))
                # preds = preds.cpu()
                # batch_y = batch_y.cpu()
                # ece_ = ece(preds,batch_y)
                # ece_ = ece_.to("cuda")
                # scheduler.step(val_loss)
                # loss = loss + ece_
                # loss = F.nll_loss(preds,batch_y)

                # Backpropagate to get the gradients
                loss.backward()

                self.optimizer.step()
                # Update progress bar with accuracy occasionally
                if batch_idx % self.print_interval == 0:
                    current_logits = self.network(batch_x)
                    current_accuracy = (current_logits.argmax(axis=1) == batch_y).float().mean()
                    progress_bar.set_postfix(loss=loss.item(), acc=current_accuracy.item())
          

    def predict_probabilities(self, x: torch.Tensor, num_sample=100) -> torch.Tensor:
        self.network.eval()

        x = x.to("cuda").view(-1,1,28,28)
        # TODO: MC_Dropout_3. Implement your MC_dropout prediction here
        # You need to sample from your trained model here multiple times
        # in order to implement Monte Carlo integration
        preds = []
        for i in range(10):
            pred = self.network(x)
            pred = F.softmax(pred,dim=-1)
            pred = pred.detach().cpu().numpy()
            preds.append(pred)

        preds = np.array(preds)

        # print("preds shape")
        # print(preds.shape)
        # print(preds)

        estimated_probability = preds.mean(axis=0)

        # print("estimated_probability shape")
        # print(estimated_probability.shape)
        # print(estimated_probability[0])

        estimated_probability = torch.from_numpy(estimated_probability)
        
        assert estimated_probability.shape == (x.shape[0], 10)  
        return estimated_probability

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# soft accuracy:
# parts of the code are taken from https://github.com/wildltr/ptranking, specifically 
# https://github.com/wildltr/ptranking/blob/master/ptranking/base/neural_utils.py

# soft binning:
# we have used parts of the code from https://github.com/wOOL/DNDT/blob/master/pytorch/demo.ipynb


class Robust_Sigmoid(torch.autograd.Function):
    ''' Aiming for a stable sigmoid operator with specified sigma '''

    @staticmethod
    def forward(ctx, input, sigma=1.0, gpu=False):
        '''
        :param ctx:
        :param input: the input tensor
        :param sigma: the scaling constant
        :return:
        '''
        x = input if 1.0 == sigma else sigma * input

        torch_half = torch.cuda.FloatTensor(
            [0.5]) if gpu else torch.FloatTensor([0.5])
        sigmoid_x_pos = torch.where(
            input > 0, 1./(1. + torch.exp(-x)), torch_half)

        exp_x = torch.exp(x)
        sigmoid_x = torch.where(input < 0, exp_x/(1.+exp_x), sigmoid_x_pos)

        grad = sigmoid_x * \
            (1. - sigmoid_x) if 1.0 == sigma else sigma * \
            sigmoid_x * (1. - sigmoid_x)
        ctx.save_for_backward(grad)

        return sigmoid_x

    @staticmethod
    def backward(ctx, grad_output):
        '''
        :param ctx:
        :param grad_output: backpropagated gradients from upper module(s)
        :return:
        '''
        grad = ctx.saved_tensors[0]

        bg = grad_output * grad  # chain rule

        return bg, None, None


#- function: robust_sigmoid-#
robust_sigmoid = Robust_Sigmoid.apply


class DECE(nn.Module):
    """
    Computes DECE loss (differentiable expected calibration error).
    """

    def __init__(self, device, num_bins, t_a, t_b):
        super(DECE, self).__init__()
        self.device = device
        self.num_bins = num_bins
        self.t_a = t_a
        self.t_b = t_b

    def one_hot(self, indices, depth):
        """
        Returns a one-hot tensor.
        This is a PyTorch equivalent of Tensorflow's tf.one_hot.
        Parameters:
        indices:  a (n_batch, m) Tensor or (m) Tensor.
        depth: a scalar. Represents the depth of the one hot dimension.
        Returns: a (n_batch, m, depth) Tensor or (m, depth) Tensor.
        """
        encoded_indicies = torch.zeros(
            indices.size() + torch.Size([depth])).to(device=self.device)
        index = indices.view(indices.size() + torch.Size([1]))
        encoded_indicies = encoded_indicies.scatter_(1, index, 1)

        return encoded_indicies

    def forward(self, input, target):
        if input.dim() > 2:
            # N,C,H,W => N,C,H*W
            input = input.view(input.size(0), input.size(1), -1)
            input = input.transpose(1, 2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))   # N,H*W,C => N*H*W,C

        # For CIFAR-10 and CIFAR-100, target.shape is [N] to begin with
        target = target.view(-1)

        predicted_probs = F.softmax(input, dim=1)

        cut_points = torch.linspace(0, 1, self.num_bins + 1)[:-1].to(device=self.device)
        W = torch.reshape(torch.linspace(1.0, self.num_bins, self.num_bins).to(device=self.device), [1, -1])
        b = torch.cumsum(-cut_points, 0)

        confidences = torch.max(predicted_probs, dim=1, keepdim=True)[0]
        h = torch.matmul(confidences, W) + b
        h = h / self.t_b

        bin_probs = F.softmax(h, dim=1)

        # smoothen the probabilities to avoid zeros
        eps = 1e-6
        bin_probs = bin_probs + eps
        # normalize the probabilities to sum to one across bins
        bin_probs = bin_probs / (1.0 + (self.num_bins + 1) * eps)

        # calculate bin confidences
        bin_confs = torch.div(bin_probs.transpose(0, 1).matmul(confidences).view(-1),
                              torch.sum(bin_probs, dim=0))
        # all-pairs approach
        batch_pred_diffs = torch.unsqueeze(predicted_probs, dim=2) - torch.unsqueeze(predicted_probs, dim=1)
        # computing pairwise differences, i.e., Sij or Sxy
        if str(self.device) == 'cpu':
            gpu = False
        else:
            gpu = True
        # using {-1.0*} may lead to a poor performance when compared with the above way;
        batch_indicators = robust_sigmoid(torch.transpose(batch_pred_diffs, dim0=1, dim1=2), self.t_a, gpu)

        # get approximated rank positions, i.e., hat_pi(x)
        ranks_all = torch.sum(batch_indicators, dim=2) + 0.5
        # the ranks go from 1 to C, with 1 being the best rank
        true_ranks = ranks_all[torch.arange(ranks_all.size(0)), target]
        accs = F.relu(2.0 - true_ranks)
        bin_accs = torch.div(bin_probs.transpose(0, 1).matmul(accs).view(-1),
                                torch.sum(bin_probs, dim=0))

        # calculate overall ECE for the whole batch
        ece = torch.sum(torch.sum(bin_probs, dim=0) * torch.abs(bin_accs - bin_confs) / bin_probs.shape[0], dim=0)
        return ece

In [11]:
def run_solution(dataset_train: torch.utils.data.Dataset, data_dir: str = os.curdir, output_dir: str = '/results/'):
    """
    Run your task 2 solution.
    This method should train your model, evaluate it, and return the trained model at the end.
    Make sure to preserve the method signature and to return your trained model,
    else the checker will fail!

    :param dataset_train: Training dataset
    :param data_dir: Directory containing the datasets
    :return: Your trained model
    """


    trainer = DropoutTrainer(dataset_train=dataset_train)


    # Train the model
    print('Training model')
    trainer.train()

    # Predict using the trained model
    print('Evaluating model on training data')
    eval_loader = torch.utils.data.DataLoader(
        dataset_train, batch_size=64, shuffle=False, drop_last=False
    )
    evaluate(trainer, eval_loader, data_dir, output_dir)

    # IMPORTANT: return your model here!
    return trainer

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load training data
data_dir = os.curdir
output_dir = os.curdir
raw_train_data = np.load(os.path.join(data_dir, 'train_data.npz'))
x_train = torch.from_numpy(raw_train_data['train_x']).to(device)
y_train = torch.from_numpy(raw_train_data['train_y']).long().to(device)
dataset_train = torch.utils.data.TensorDataset(x_train, y_train)
dataset_test = torch.utils.data.TensorDataset(x_train, y_train)
# Run actual solution
trainer = run_solution(dataset_train, data_dir=data_dir, output_dir=output_dir)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Training model


 14%|█▍        | 14/100 [00:32<02:13,  1.56s/it, acc=0.406, loss=2.28]

In [ ]:
trainer = run_solution(dataset_train, data_dir=data_dir, output_dir=output_dir)

In [ ]:
trainer.network = trainer.network.cpu()
torch.save(trainer.network.state_dict(),"trainer.pth")
# torch.save(trainer.state_dict(), 'trainer.pth')